# An accurate boundary-integral formulation for the calculation of electrostatic forces with an implicit-solvent model

### Import libraries

In [1]:
import os
import numpy as np
import pbj  # 

### Import custom-functions 

In [2]:
#magnitud of a vector numpy
def mag(v):
    return np.sqrt(np.dot(v, v))

#Translation pqr
def pqr_translate(directory, protein_pqr, distance):
    'Translate along distance vector the protein .pqr file'

    protein_pqr_d = protein_pqr + '_d' + str(distance[2]) #new name with z-distance
    mesh_pqr_path = '{}/{}.pqr'.format(directory,protein_pqr)
    mesh_pqr_path_out = '{}/{}.pqr'.format(directory,protein_pqr_d)

    pqr_file = open(mesh_pqr_path, 'r')
    pqr_data = pqr_file.read().split('\n')
    pqr_t_file = open(mesh_pqr_path_out, 'w')
    for line in pqr_data:
        line = line.split()
        if len(line) == 0 or line[0] != 'ATOM':
            continue
        tx,ty,tz = distance
        x_new = float(line[5])+ float(tx)
        y_new = float(line[6])+ float(ty)
        z_new = float(line[7])+ float(tz)
        text_template = '{} {}  {}   {} {}        {: 1.3f}  {: 1.3f}  {: 1.3f}  {}  {} \n'
        pqr_t_file.write(text_template.format(line[0],line[1],line[2],line[3],line[4],x_new,y_new,z_new,line[8],line[9]))

    pqr_file.close() 
    pqr_t_file.close()

    return None

In [20]:
pqr_translate('/home/ianaddisonsmith/pbj/notebooks/tutorials/pqrs','1brs_BE_barstar',[0,0,1000])

## 1lyz

### Simulation with different mesh density (vert/Ang2)

In [7]:
mesh_vertA2_sim = [2,4,8,16]

# 1lyz simulation
for mesh_value in mesh_vertA2_sim:
    
    f = open("results_pbj//1lyz_forces_to_latex.txt", "a")
    f_terms = open("results_pbj//1lyz_forces_terms_to_latex.txt", "a")
    print('-----------------------------------')
    print('mesh value: ', mesh_value)
    
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//1lyz_amber.pqr", mesh_density=mesh_value)
    simulation.operator_assembler = 'fmm'
    protein.operator_assembler = "fmm"
    protein.rhs_constructor = "fmm"
    
    simulation.add_solute(protein)
    simulation.calculate_solvation_energy()

    # Forces with energy functional
    print('Energy functional')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full = protein.results['f_solv']
    print(protein.results['f_solv'])

    # Forces with maxwell tensor
    print('Maxwell tensor')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein.results['f_solv']
    print(protein.results['f_solv'])
    
    f.write('%d & %.4f & %.4f & %.6f  \\\ \n'%(mesh_value,\
                                               mag(f_solv_full),\
                                               mag(f_solv_fast), \
                                               protein.results['solvation_energy']))
    f_terms.write('%d & %.4f & %.4f & %.4f & %.6f  \\\ \n'%(mesh_value,\
                                                     mag(protein.results["f_qf"]),\
                                                     mag(protein.results['f_db']),\
                                                     mag(protein.results['f_ib']),\
                                                     protein.results['solvation_energy']))
    f.close()
    f_terms.close()

-----------------------------------
mesh value:  2
Energy functional
[ 4.7586928   0.21258493 -2.21983287]
Maxwell tensor
[-0.58577772 -0.0808591   0.19743213]
-----------------------------------
mesh value:  4


## 1bpi

### Simulation with different mesh density (vert/Ang) (2,4,8,16)

In [9]:
mesh_vertA2_sim = [2,4,8,16]

# 1lyz simulation
for mesh_value in mesh_vertA2_sim:
    f = open("results_pbj//1bpi_forces_to_latex.txt", "a")
    f_terms = open("results_pbj//1bpi_forces_terms_to_latex.txt", "a")
    print('-----------------------------------')
    print('mesh value: ', mesh_value)
    
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//1bpi.pqr", mesh_density=mesh_value)
    simulation.operator_assembler = 'fmm'
    protein.operator_assembler = "fmm"
    protein.rhs_constructor = "fmm"
    
    simulation.add_solute(protein)
    simulation.calculate_solvation_energy()

    # Forces with energy functional
    print('Energy functional')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full = protein.results['f_solv']
    print(protein.results['f_solv'])

    # Forces with maxwell tensor
    print('Maxwell tensor')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein.results['f_solv']
    print(protein.results['f_solv'])
    
    f.write('%d & %.4f & %.4f & %.2f  \\\ \n'%(mesh_value,\
                                               mag(f_solv_full),\
                                               mag(f_solv_fast), \
                                               protein.results['solvation_energy']))
    f_terms.write('%d & %.4f & %.4f & %.4f & %.2f  \\\ \n'%(mesh_value,\
                                                     mag(protein.results["f_qf"]),\
                                                     mag(protein.results['f_db']),\
                                                     mag(protein.results['f_ib']),\
                                                     protein.results['solvation_energy']))
    f.close()
    f_terms.close()

-----------------------------------
mesh value:  2


 <<INFO>> Starting NanoShaper 0.7.8
 <<INFO>> Loading atoms....
 <<INFO>> Read 892 atoms
 <<INFO>> Geometric baricenter ->  30.181 10.2855 0.166
 <<INFO>> Grid is 81
 <<INFO>> MAX 66.5446 46.6491 36.5296
 <<INFO>> MIN -6.18264 -26.0781 -36.1976
 <<INFO>> Perfil 50 %
 <<INFO>> Rmaxdim 37.138
 <<INFO>> Allocating memory...ok!
 <<INFO>> Initialization completed
 <<INFO>> Adjusting self intersection grid 
 <<INFO>> Self intersection grid is (before) 26
 <<INFO>> Self intersection grid is 16
 <<INFO>> Allocating self intersection grid....ok!
 <<INFO>> Computing alpha shape complex....ok!
 <<INFO>> Checking 128 probes for self intersections...ok!
 <<INFO>> Surface build-up time.. 0 [s]
 <<INFO>> Probe Radius value 1.4
 <<INFO>> Number of ses cells -> 2725
 <<INFO>> Number of del_point cells -> 430
 <<INFO>> Number of regular del_edge cells -> 1337
 <<INFO>> Number of singular del_edge cells -> 0
 <<INFO>> Number of regular del_facet cells 

66 warnings generated.
66 warnings generated.


formulacion con la suma de terminos
[ 0.21981494 -0.01863     0.1320418 ]
formulacion nueva
[-0.01856452 -0.02606556 -0.02945963]
-----------------------------------
mesh value:  8


 <<INFO>> Starting NanoShaper 0.7.8
 <<INFO>> Loading atoms....
 <<INFO>> Read 892 atoms
 <<INFO>> Geometric baricenter ->  30.181 10.2855 0.166
 <<INFO>> Grid is 171
 <<INFO>> MAX 67.1375 47.242 37.1225
 <<INFO>> MIN -6.77552 -26.671 -36.7905
 <<INFO>> Perfil 50 %
 <<INFO>> Rmaxdim 37.138
 <<INFO>> Allocating memory...ok!
 <<INFO>> Initialization completed
 <<INFO>> Adjusting self intersection grid 
 <<INFO>> Self intersection grid is (before) 26
 <<INFO>> Self intersection grid is 16
 <<INFO>> Allocating self intersection grid....ok!
 <<INFO>> Computing alpha shape complex....ok!
 <<INFO>> Checking 128 probes for self intersections...ok!
 <<INFO>> Surface build-up time.. 0 [s]
 <<INFO>> Probe Radius value 1.4
 <<INFO>> Number of ses cells -> 2725
 <<INFO>> Number of del_point cells -> 430
 <<INFO>> Numbe

## Two centered-charge (2q) unit spheres

### Simulation at distance 3 Ang between centers with variable grid mesh (vert/$\AA ^2$)

In [3]:
mesh_vertA2_sim = [2,4,8,16,32,64]

for mesh_value in mesh_vertA2_sim:
    
    # open a file to write the results
    f = open("results_pbj//sphere_r1q2_forces_to_latex.txt", "a")
    f_terms = open("results_pbj//sphere_r1q2_forces_terms_to_latex.txt", "a")
    print('-----------------------------------')
    print('mesh value: ', mesh_value)

    #Solvation energy alone
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=mesh_value, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.calculate_solvation_energy()
    solv_energy_alone = protein.results['solvation_energy']

    #Solvation energy complex
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=mesh_value, mesh_generator="msms")
    protein2 = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2_d3.pqr", mesh_density=mesh_value, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.calculate_solvation_energy()
    solv_energy_complex = protein.results['solvation_energy'] + protein2.results['solvation_energy']
    solv_energy_binding = solv_energy_complex - 2*solv_energy_alone

    #Forces calculation   
    print('Energy functional with fdb approx')
    simulation.calculate_solvation_forces(force_formulation='energy_functional',fdb_approx=True)
    f_solv_full_app = protein2.results['f_solv']
    fdb_app = protein2.results['f_db']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Energy functional with fdb exact')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full_ex = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Maxwell tensor')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])
    f.write('%d & %.4f & %.4f & %.4f & %.4f  \\\ \n'%(mesh_value,\
                                               f_solv_full_app[0], \
                                                f_solv_full_ex[0], \
                                               f_solv_fast[0],\
                                               solv_energy_binding))
    
    f_terms.write('%d & %.4f & %.4f & %.4f & %.4f & %.2f  \\\ \n'%(mesh_value,\
                                                     mag(protein2.results["f_qf"]),\
                                                     mag(protein2.results['f_db']),\
                                                     mag(fdb_app),\
                                                     mag(protein2.results['f_ib']),\
                                                     protein.results['solvation_energy']))

    f.close()
    f_terms.close()

-----------------------------------
mesh value:  2
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 2 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  1.000
PARAM  Probe_radius  1.400 density  2.000 hdensity  3.000
REDUCED SURFACE ...
    1 free vertex (vertices) found
    RS component  #faces  #edges #free_edges  #vertices   genus
         0           0        1          0          1      1
    Time Reduced Surface real: 0.00 user: 0.00 sys: 0.00
ANALYTICAL SOLVENT EXCLUDED SURFACE...
    Component   0
        Time Surface

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 2 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  1.000
PARAM  Probe_radius  1.400 density  2.000 hdensity  3.000
REDUCED SURFACE ...
    1 free vertex (vertices) found
    RS component  #faces  #edges #free_edges  #vertices   genus
         0           0        1          0          1      1
    Time Reduced Surface real: 0.00 user: 0.00 sys: 0.00
ANALYTICAL SOLVENT EXCLUDED SURFACE...
    Component   0
        Time Surface real: 0.00 user: 0.00 sys: 0.00
    SES comp.  #fa

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Energy functional with fdb approx


...

Energy functional with fdb approx
[-1.86045210e+00  1.26919694e-02 -3.19864244e-07]
[1.86050957e+00 1.27038933e-02 1.04677328e-05]
Energy functional with fdb exact
[-1.94375640e+00  1.27733826e-02  2.21348059e-06]
[1.94381548e+00 1.27851062e-02 1.08608230e-05]
Maxwell tensor
[-1.93954508e+00 -5.32930622e-04 -3.31576538e-06]
[ 1.93953449e+00 -5.21940937e-04  3.52951151e-06]
-----------------------------------
mesh value:  64
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 64 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 64 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  1.000
PARAM  Probe_radius  1.400 density 64.000 hdensity  3.000
REDUCED SURFACE ...
    1 free vertex (vertices) found
    RS component  #faces  #edges #free_edges  #vertices   genus
         0           0        1          0          1      1
    Time Reduced Surface real: 0.00 user: 0.00 sys: 0.00
ANALYTICAL SOLVENT EXCLUDED SURFACE...
    Component   0
        Time Surface real: 0.00 user: 0.00 sys: 0.00
    SES comp.  #f

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Energy functional with fdb approx
[-1.85769543e+00  8.56940440e-05  1.09006169e-05]
[1.85767209e+00 8.74157981e-05 3.70112367e-06]
Energy functional with fdb exact
[-1.94042462e+00  6.07916044e-05  1.16005883e-05]
[1.94039562e+00 6.36662164e-05 4.04490304e-06]
Maxwell tensor
[-1.93837568e+00 -3.61327021e-06  2.35311860e-06]
[ 1.93838589e+00 -1.62455630e-06 -5.65836507e-06]


In [4]:
pbj.electrostatics.solute.bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 8 # 4 is default
mesh_vertA2_sim = [2,4,8,16,32,64]

for mesh_value in mesh_vertA2_sim:
    
    # open a file to write the results
    f = open("results_pbj//sphere_r1q2_forces_to_latex_param.txt", "a")
    f_terms = open("results_pbj//sphere_r1q2_forces_terms_to_latex_param.txt", "a")
    print('-----------------------------------')
    print('mesh value: ', mesh_value)

    #Solvation energy alone
    simulation = pbj.electrostatics.Simulation()
    
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=mesh_value, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.gmres_tolerance = 1e-8
    simulation.gmres_restart = 3000
    simulation.gmres_max_iterations = 3000
    simulation.calculate_solvation_energy()
    solv_energy_alone = protein.results['solvation_energy']

    #Solvation energy complex
    simulation = pbj.electrostatics.Simulation()
    simulation.gmres_tolerance = 1e-8
    simulation.gmres_restart = 3000
    simulation.gmres_max_iterations = 3000
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=mesh_value, mesh_generator="msms")
    protein2 = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2_d3.pqr", mesh_density=mesh_value, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.calculate_solvation_energy()
    solv_energy_complex = protein.results['solvation_energy'] + protein2.results['solvation_energy']
    solv_energy_binding = solv_energy_complex - 2*solv_energy_alone

    #Forces calculation   
    print('Energy functional with fdb approx')
    simulation.calculate_solvation_forces(force_formulation='energy_functional',fdb_approx=True)
    f_solv_full_app = protein2.results['f_solv']
    fdb_app = protein2.results['f_db']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Energy functional with fdb exact')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full_ex = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Maxwell tensor')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])
    f.write('%d & %.4f & %.4f & %.4f & %.4f  \\\ \n'%(mesh_value,\
                                               f_solv_full_app[0], \
                                                f_solv_full_ex[0], \
                                               f_solv_fast[0],\
                                               solv_energy_binding))
    
    f_terms.write('%d & %.4f & %.4f & %.4f & %.4f & %.2f  \\\ \n'%(mesh_value,\
                                                     mag(protein2.results["f_qf"]),\
                                                     mag(protein2.results['f_db']),\
                                                     mag(fdb_app),\
                                                     mag(protein2.results['f_ib']),\
                                                     protein.results['solvation_energy']))

    f.close()
    f_terms.close()

-----------------------------------
mesh value:  2
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 2 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  1.000
PARAM  Probe_radius  1.400 density  2.000 hdensity  3.000
REDUCED SURFACE ...
    1 free vertex (vertices) found
    RS component  #faces  #edges #free_edges  #vertices   genus
         0           0        1          0          1      1
    Time Reduced Surface real: 0.00 user: 0.00 sys: 0.00
ANALYTICAL SOLVENT EXCLUDED SURFACE...
    Component   0
        Time Surface

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 2 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  1.000
PARAM  Probe_radius  1.400 density  2.000 hdensity  3.000
REDUCED SURFACE ...
    1 free vertex (vertices) found
    RS component  #faces  #edges #free_edges  #vertices   genus
         0           0        1          0          1      1
    Time Reduced Surface real: 0.00 user: 0.00 sys: 0.00
ANALYTICAL SOLVENT EXCLUDED SURFACE...
    Component   0
        Time Surface real: 0.00 user: 0.00 sys: 0.00
    SES comp.  #fa

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Energy functional with fdb approx
[-1.89439724e+00  1.25855161e-02 -1.12835138e-06]
[ 1.89437741e+00  1.25868871e-02 -5.84467622e-07]
Energy functional with fdb exact
[-1.92031769e+00  1.52466320e-02  8.44188815e-08]
[ 1.92030336e+00  1.52491742e-02 -1.44178390e-06]
Maxwell tensor
[-1.86068037e+00 -5.60362889e-03  2.21657796e-08]
[ 1.86066475e+00 -5.60162864e-03 -1.46317390e-06]
-----------------------------------
mesh value:  4
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 4 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 

...

Energy functional with fdb approx
[-1.85979898e+00  1.26981413e-02  3.80779563e-06]
[1.85985492e+00 1.26990246e-02 2.55279883e-06]
Energy functional with fdb exact
[-1.94290797e+00  1.27793672e-02  6.04446745e-06]
[1.94296549e+00 1.27797769e-02 3.29038661e-06]
Maxwell tensor
[-1.93868660e+00 -5.29559910e-04  1.04537888e-06]
[ 1.93867394e+00 -5.29185941e-04 -4.07982287e-06]
-----------------------------------
mesh value:  64
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 64 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr -of /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2 -p 1.4 -d 64 -no_header
MSMS 2.6.1 started on ian
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
Atom 2 ignored, radius = 0.000000
Atom 3 ignored, radius = 0.000000
Atom 4 ignored, radius = 0.000000
INPUT  /home/ian/Desktop/paper_PBforces/mesh_temp/sphere_point_r1q2.xyzr 1 spheres 0 collision only, radii  1.000 to  1.000
PARAM  Probe_radius  1.400 density 64.000 hdensity  3.000
REDUCED SURFACE ...
    1 free vertex (vertices) found
    RS component  #faces  #edges #free_edges  #vertices   genus
         0           0        1          0          1      1
    Time Reduced Surface real: 0.00 user: 0.00 sys: 0.00
ANALYTICAL SOLVENT EXCLUDED SURFACE...
    Component   0
        Time Surface real: 0.00 user: 0.00 sys: 0.00
    SES comp.  #f

/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
/home/ian/anaconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Energy functional with fdb approx
[-1.85962883e+00  8.75980297e-05  7.74281868e-06]
[1.85960363e+00 8.80518074e-05 7.60059909e-06]
Energy functional with fdb exact
[-1.94250326e+00  6.18426095e-05  8.66081569e-06]
[1.94247244e+00 6.32213607e-05 7.90347837e-06]
Maxwell tensor
[-1.94047234e+00 -3.65489604e-06 -9.73108744e-07]
[ 1.94048124e+00 -3.26707722e-06 -2.04855250e-06]


### Virtual displacements

In [41]:
# Trabajo virtual

mesh_vertA2_sim = [2,4,8,16]

for mesh_value in mesh_vertA2_sim:
    print('-----------------------------------')
    print('mesh value: ', mesh_value)

    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs\\sphere_point_r1q2.pqr", mesh_density=mesh_value, mesh_generator="msms")
    protein2 = pbj.electrostatics.Solute("pqrs\\sphere_point_r1q2_d3+h0.001.pqr", mesh_density=mesh_value, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.calculate_solvation_energy()
    solv_energyh_adelante = protein2.results['solvation_energy']

    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs\\sphere_point_r1q2.pqr", mesh_density=mesh_value, mesh_generator="msms")
    protein2 = pbj.electrostatics.Solute("pqrs\\sphere_point_r1q2_d3-h0.001.pqr", mesh_density=mesh_value, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.calculate_solvation_energy()
    solv_energyh_atras = protein2.results['solvation_energy']

    print('La fuerza en direccion x: %f kcal/molA usando desplazamiento virtuales y h=0.001' %float(-(solv_energyh_adelante - solv_energyh_atras) / 0.002))

-----------------------------------
mesh value:  2
C:\Users\ianad\Desktop\pbj\pbj\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\sphere_point_r1q1.xyzr -of c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\sphere_point_r1q1 -p 1.4 -d 2 -no_header
C:\Users\ianad\Desktop\pbj\pbj\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\001.xyzr -of c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\001 -p 1.4 -d 2 -no_header
C:\Users\ianad\Desktop\pbj\pbj\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\sphere_point_r1q1.xyzr -of c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\sphere_point_r1q1 -p 1.4 -d 2 -no_header
C:\Users\ianad\Desktop\pbj\pbj\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\001.xyzr -of c:\Users\ianad\Desktop\pbj\pbj\mesh_temp\001 -p 1.4 -d 2 -no_header
La fuerza en direccion x: 0.234901 kcal/molA usando desplazamiento virtuales y h=0.001
------------------

### Simulation at several distances between centers 

In [24]:
dist_sim = [3]
for dist in dist_sim:
    
    # open a file to write the results
    f = open("results_pbj//sphere_r1q2_forces_to_latex_dist.txt", "a")
    f_terms = open("results_pbj//sphere_r1q2_forces_terms_to_latex_dist.txt", "a")
    print('-----------------------------------')
    print('Distance: %d A'%(dist))

    #Solvation energy alone
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=32, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.calculate_solvation_energy()
    solv_energy_alone = protein.results['solvation_energy']

    #Solvation energy complex
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=32, mesh_generator="msms")
    protein2 = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2_d%d.pqr"%(dist), mesh_density=32, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.calculate_solvation_energy()
    solv_energy_complex = protein.results['solvation_energy'] + protein2.results['solvation_energy']
    solv_energy_binding = solv_energy_complex - 2*solv_energy_alone

    #Forces calculation   
    print('Energy functional with fdb approx')
    simulation.calculate_solvation_forces(force_formulation='energy_functional', fdb_approx=True)
    f_solv_full_app = protein2.results['f_solv']
    fdb_app = protein2.results['f_db']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Energy functional with fdb exact')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full_ex = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Maxwell tensor formulation')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    f.write('%d & %.4f & %.4f & %.4f & %.4f  \\\ \n'%(dist,\
                                               f_solv_full_app[0], \
                                                f_solv_full_ex[0], \
                                               f_solv_fast[0],\
                                               solv_energy_binding))
    
    f_terms.write('%d & %.4f & %.4f & %.4f & %.4f & %.2f  \\\ \n'%(dist,\
                                                     protein2.results["f_qf"][0],\
                                                     protein2.results['f_db'][0],\
                                                     fdb_app[0],\
                                                     protein2.results['f_ib'][0],\
                                                     protein2.results['solvation_energy']))

    f.close()
    f_terms.close()

-----------------------------------
Distance: 3 A
C:\Users\ianad\Desktop\paper_PBforces\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2.xyzr -of c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2 -p 1.4 -d 32 -no_header
C:\Users\ianad\Desktop\paper_PBforces\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2.xyzr -of c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2 -p 1.4 -d 32 -no_header
C:\Users\ianad\Desktop\paper_PBforces\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2_d3.xyzr -of c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2_d3 -p 1.4 -d 32 -no_header
Dielectric approx formulation
[-1.86156753e+00  1.28740192e-02 -2.06398716e-05]
[ 1.86149419e+00  1.28711565e-02 -2.35204130e-05]
Dielectric exact formulation
[-1.94492624e+00  1.30958594e-02 -2.55019346

## Barnase-Barstar complex

### Simulacion barnase alone

In [12]:
mesh_vertA2_sim = [2,4,8,16]

for mesh_value in mesh_vertA2_sim:
    f = open("results_pbj//barnase_forces_to_latex.txt", "a")
    print('-----------------------------------')
    print('mesh value: ', mesh_value)
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//1brs_BE_barnase.pqr", mesh_density=mesh_value)
    simulation.operator_assembler = 'fmm'
    protein.operator_assembler = "fmm"
    protein.rhs_constructor = "fmm"
    simulation.add_solute(protein)
    simulation.calculate_solvation_energy()

    print('Energy functional')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full = protein.results['f_solv']
    print(protein.results['f_solv'])

    print('Maxwell tensor ')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein.results['f_solv']
    print(protein.results['f_solv'])
    f.write('%d & %.4f & %.4f & %.6f  \\\ \n'%(mesh_value,mag(f_solv_full), mag(f_solv_fast), protein.results['solvation_energy']))
    f.close()

-----------------------------------
mesh value:  2


 <<INFO>> Starting NanoShaper 0.7.8
 <<INFO>> Loading atoms....
 <<INFO>> Read 1727 atoms
 <<INFO>> Geometric baricenter ->  -4.2595 23.3285 12.6551
 <<INFO>> Grid is 101
 <<INFO>> MAX 41.195 68.783 58.1096
 <<INFO>> MIN -49.714 -22.126 -32.7994
 <<INFO>> Perfil 50 %
 <<INFO>> Rmaxdim 46.327
 <<INFO>> Allocating memory...ok!
 <<INFO>> Initialization completed
 <<INFO>> Adjusting self intersection grid 
 <<INFO>> Self intersection grid is (before) 33
 <<INFO>> Self intersection grid is 32
 <<INFO>> Allocating self intersection grid....ok!
 <<INFO>> Computing alpha shape complex....ok!
 <<INFO>> Checking 236 probes for self intersections...ok!
 <<INFO>> Surface build-up time.. 0 [s]
 <<INFO>> Probe Radius value 1.4
 <<INFO>> Number of ses cells -> 4484
 <<INFO>> Number of del_point cells -> 702
 <<INFO>> Number of regular del_edge cells -> 2203
 <<INFO>> Number of singular del_edge cells -> 1
 <<INFO>> Number of regular del_facet cells

### Simulation barstar alone

In [13]:
mesh_vertA2_sim = [2,4,8,16]

for mesh_value in mesh_vertA2_sim:
    f = open("results_pbj//barstar_forces_to_latex.txt", "a")
    print('-----------------------------------')
    print('mesh value: ', mesh_value)
    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//1brs_BE_barstar.pqr", mesh_density=mesh_value)
    simulation.operator_assembler = 'fmm'
    protein.operator_assembler = "fmm"
    protein.rhs_constructor = "fmm"
    simulation.add_solute(protein)
    simulation.calculate_solvation_energy()

    print('Energy functional')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full = protein.results['f_solv']
    print(protein.results['f_solv'])

    print('Maxwell tensor')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein.results['f_solv']
    print(protein.results['f_solv'])
    f.write('%d & %.4f & %.4f & %.6f  \\\ \n'%(mesh_value,mag(f_solv_full), mag(f_solv_fast), protein.results['solvation_energy']))
    f.close()

-----------------------------------
mesh value:  2


 <<INFO>> Starting NanoShaper 0.7.8
 <<INFO>> Loading atoms....
 <<INFO>> Read 1381 atoms
 <<INFO>> Geometric baricenter ->  -2.1761 13.3966 34.994
 <<INFO>> Grid is 85
 <<INFO>> MAX 36.0057 51.5784 73.1758
 <<INFO>> MIN -40.3579 -24.7852 -3.18782
 <<INFO>> Perfil 50 %
 <<INFO>> Rmaxdim 38.248
 <<INFO>> Allocating memory...ok!
 <<INFO>> Initialization completed
 <<INFO>> Adjusting self intersection grid 
 <<INFO>> Self intersection grid is (before) 27
 <<INFO>> Self intersection grid is 16
 <<INFO>> Allocating self intersection grid....ok!
 <<INFO>> Computing alpha shape complex....ok!
 <<INFO>> Checking 184 probes for self intersections...ok!
 <<INFO>> Surface build-up time.. 0 [s]
 <<INFO>> Probe Radius value 1.4
 <<INFO>> Number of ses cells -> 3360
 <<INFO>> Number of del_point cells -> 536
 <<INFO>> Number of regular del_edge cells -> 1658
 <<INFO>> Number of singular del_edge cells -> 0
 <<INFO>> Number of regular del_facet cel

### Simulacion barstar-barnase complex with 8 vert/$\AA ^2$ at several distances

In [ ]:
# Metodos Bem

distances = [9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,100,1000]

for dist in distances:
    # open a file to write the results
    f = open("results_pbj//barnase-barstar_forces_to_latex_mesh8.txt", "a")
    f_terms = open("results_pbj//barnase-barstar_forces_terms_to_latex_mesh8.txt", "a")
    print('-----------------------------------')
    print('Distance: ', dist)

    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//1brs_BE_barnase.pqr", mesh_density=8)
    protein2 = pbj.electrostatics.Solute("pqrs//1brs_BE_barstar_d%d.pqr"%(dist), mesh_density=8)
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.operator_assembler = 'fmm'
    protein.operator_assembler = "fmm"
    protein.rhs_constructor = "fmm"
    protein2.operator_assembler = "fmm"
    protein2.rhs_constructor = "fmm"
    
    print('Energy functional with fdb approx')
    simulation.calculate_solvation_energy()
    simulation.calculate_solvation_forces(force_formulation='energy_functional',fdb_approx=True)
    binding_energy = (protein.results['solvation_energy'] + protein2.results['solvation_energy'])-(-312.703404+-395.316315)
    f_solv_full1_app = protein.results['f_solv']
    f_solv_full2_app = protein2.results['f_solv']
    fdb_app_1 = protein.results['f_db']
    fdb_app_2 = protein2.results['f_db']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Energy functional with fdb exact')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full1 = protein.results['f_solv']
    f_solv_full2 = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Maxwell tensor')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast1 = protein.results['f_solv']
    f_solv_fast2 = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])
    f.write('%d & %.4f & %.4f & %.4f & %.4f & %.4f & %.4f & %.6f & %.6f & %.6f  \\\ \n'%(dist,f_solv_full1[2], f_solv_full1_app[2], f_solv_fast1[2],\
                                                                                f_solv_full2[2], f_solv_full2_app[2], f_solv_fast2[2],\
                                                                                binding_energy,\
                                                                                protein.results['solvation_energy'],protein2.results['solvation_energy']))

    f_terms.write('%d & %.4f & %.4f & %.4f & %.4f & & %.4f & %.4f & %.4f & %.4f & %.6f  \\\ \n'%(dist,\
                                                     protein.results["f_qf"][2],protein.results['f_db'][2],fdb_app_1[2],\
                                                     protein.results['f_ib'][2],protein2.results["f_qf"][2],protein2.results['f_db'][2],fdb_app_2[2],\
                                                     protein2.results['f_ib'][2],protein.results['solvation_energy']))

    
    f.close()
    f_terms.close()

### Simulacion barstar-barnase complex with 4 vert/$\AA ^2$ at several distances

In [8]:
# Metodos Bem

distances = [9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,100]

for dist in distances:
    # open a file to write the results
    f = open("results_pbj//barnase-barstar_forces_to_latex_mesh4.txt", "a")
    f_terms = open("results_pbj//barnase-barstar_forces_terms_to_latex_mesh4.txt", "a")
    print('-----------------------------------')
    print('Distance: ', dist)

    simulation = pbj.electrostatics.Simulation()
    protein = pbj.electrostatics.Solute("pqrs//1brs_BE_barnase.pqr", mesh_density=4)
    protein2 = pbj.electrostatics.Solute("pqrs//1brs_BE_barstar_d%d.pqr"%(dist), mesh_density=4)
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.operator_assembler = 'fmm'
    protein.operator_assembler = "fmm"
    protein.rhs_constructor = "fmm"
    protein2.operator_assembler = "fmm"
    protein2.rhs_constructor = "fmm"
    
    print('Energy functional with fdb approx')
    simulation.calculate_solvation_energy()
    simulation.calculate_solvation_forces(force_formulation='energy_functional',fdb_approx=True)
    binding_energy = (protein.results['solvation_energy'] + protein2.results['solvation_energy'])-(-318.510721+-401.969264)
    f_solv_full1_app = protein.results['f_solv']
    f_solv_full2_app = protein2.results['f_solv']
    fdb_app_1 = protein.results['f_db']
    fdb_app_2 = protein2.results['f_db']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Energy functional with fdb exact')
    simulation.calculate_solvation_forces(force_formulation='energy_functional')
    f_solv_full1 = protein.results['f_solv']
    f_solv_full2 = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Maxwell tensor')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast1 = protein.results['f_solv']
    f_solv_fast2 = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])
    f.write('%d & %.4f & %.4f & %.4f & %.4f & %.4f & %.4f & %.6f & %.6f & %.6f  \\\ \n'%(dist,f_solv_full1[2], f_solv_full1_app[2], f_solv_fast1[2],\
                                                                                f_solv_full2[2], f_solv_full2_app[2], f_solv_fast2[2],\
                                                                                binding_energy,\
                                                                                protein.results['solvation_energy'],protein2.results['solvation_energy']))

    f_terms.write('%d & %.4f & %.4f & %.4f & %.4f & & %.4f & %.4f & %.4f & %.4f & %.6f  \\\ \n'%(dist,\
                                                     protein.results["f_qf"][2],protein.results['f_db'][2],fdb_app_1[2],\
                                                     protein.results['f_ib'][2],protein2.results["f_qf"][2],protein2.results['f_db'][2],fdb_app_2[2],\
                                                     protein2.results['f_ib'][2],protein.results['solvation_energy']))

    
    f.close()
    f_terms.close()

-----------------------------------
Distance:  20


 <<INFO>> Starting NanoShaper 0.7.8
 <<INFO>> Loading atoms....
 <<INFO>> Read 1727 atoms
 <<INFO>> Geometric baricenter ->  -4.2595 23.3285 12.6551
 <<INFO>> Grid is 213
 <<INFO>> MAX 41.8275 69.4155 58.7421
 <<INFO>> MIN -50.3465 -22.7585 -33.4319
 <<INFO>> Perfil 50 %
 <<INFO>> Rmaxdim 46.327
 <<INFO>> Allocating memory...ok!
 <<INFO>> Initialization completed
 <<INFO>> Adjusting self intersection grid 
 <<INFO>> Self intersection grid is (before) 33
 <<INFO>> Self intersection grid is 32
 <<INFO>> Allocating self intersection grid....ok!
 <<INFO>> Computing alpha shape complex....ok!
 <<INFO>> Checking 236 probes for self intersections...ok!
 <<INFO>> Surface build-up time.. 0 [s]
 <<INFO>> Probe Radius value 1.4
 <<INFO>> Number of ses cells -> 4484
 <<INFO>> Number of del_point cells -> 702
 <<INFO>> Number of regular del_edge cells -> 2203
 <<INFO>> Number of singular del_edge cells -> 1
 <<INFO>> Number of regular del_facet ce

/home/ianaddisonsmith/miniconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:274: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
/home/ianaddisonsmith/miniconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:274: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
/home/ianaddisonsmith/miniconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:274: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
/home/ianaddisonsmith/miniconda3/envs/pbj_dev/lib/python3.9/site-packages/pyopencl/__init__.py:274: CompilerWarning: Non-empty compiler output encountered

[ 0.17116385 -0.21016746  0.15257971]
[-0.0055383  -0.22701884 -0.03349448]
formulacion nueva
[0.005641   0.03115268 0.01076407]
[-0.01153863  0.01164596 -0.01454281]
-----------------------------------
Distance:  100


 <<INFO>> Starting NanoShaper 0.7.8
 <<INFO>> Loading atoms....
 <<INFO>> Read 1727 atoms
 <<INFO>> Geometric baricenter ->  -4.2595 23.3285 12.6551
 <<INFO>> Grid is 213
 <<INFO>> MAX 41.8275 69.4155 58.7421
 <<INFO>> MIN -50.3465 -22.7585 -33.4319
 <<INFO>> Perfil 50 %
 <<INFO>> Rmaxdim 46.327
 <<INFO>> Allocating memory...ok!
 <<INFO>> Initialization completed
 <<INFO>> Adjusting self intersection grid 
 <<INFO>> Self intersection grid is (before) 33
 <<INFO>> Self intersection grid is 32
 <<INFO>> Allocating self intersection grid....ok!
 <<INFO>> Computing alpha shape complex....ok!
 <<INFO>> Checking 236 probes for self intersections...ok!
 <<INFO>> Surface build-up time.. 0 [s]
 <<INFO>> Probe Radius value 1.4
 <<INFO>> Number of ses cells -> 4484
 <<INFO>> Numbe